## 共通処理

In [ ]:
# 必要なライブラリをインストール
!pip install -qU "huggingface_hub[cli]" datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.6 MB/s eta 0:00:00


In [ ]:
# Hugging Faceのアカウント名の設定
# user_id = input("Hugging Faceのアカウント名を入力してください:")
# organization_name = input("Hugging Faceの組織名を入力してください:")

# 個人設定
user_id = "ichi234"
organization_name = "geniacllm"

In [ ]:
# Hugging Faceに接続
# こちらのページを https://huggingface.co/docs/hub/en/security-tokens を参考に進めてください
# 1. 案内に従いHugging Faceにログインし、Access Tokenを作成（Nameはなんでも良く、RoleはWriteにする）。Tokenをコピーする。
# 2. Colabに戻り「Enter your token」にtokenを入力（Notebookの出力をクリックすると入力できます）
# 3. 「Add token as git credential?」に「n」を入力
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## kunishou/jp-effective-instructions

### データ加工・保存（ローカル）

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import math
import os
import json
import subprocess

original_dataset_user = "kunishou"
original_dataset_name = "jp-effective-instructions"
up_version = 1  # データセットのバージョン
output_dir = original_dataset_name

# データセットの読み込み読み込み
dataset = load_dataset(f"{original_dataset_user}/{original_dataset_name}")

print(dataset)

def unify_format(example):
    # inputカラムに'nan'が含まれている場合は空の文字列に置き換える
    if example["input"] == "nan":
        example["input"] = ""
    return example

unified_dataset = {}
# データセットに対してフォーマットの統一を適用
for split in dataset.keys():
    unified_split = dataset[split].map(unify_format)
    unified_dataset[split] = unified_split

# 統一されたデータセットをJSON形式で保存
if not os.path.exists(original_dataset_name):
    os.makedirs(original_dataset_name)

for split in unified_dataset.keys():
    json_path = f"{output_dir}/{original_dataset_name}_{split}.json"
    data = unified_dataset[split]
    # データをリストとして保存
    data_list = [data[i] for i in range(len(data))]
    # 項目のキーの順序を指定して保存
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"新しいJSON形式のデータセットが作成されました: {original_dataset_name}")

Generating train split:   0%|          | 0/5395 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'input', 'instruction', 'category', 'output'],
        num_rows: 5395
    })
})


Map:   0%|          | 0/5395 [00:00<?, ? examples/s]

新しいJSON形式のデータセットが作成されました: jp-effective-instructions


### HuggingFaceへのアップロード

In [ ]:
# コマンドを構築
dataset_name = f"{organization_name}/{original_dataset_name}-unified-v{up_version}"
upload_command = f"huggingface-cli upload --repo-type=dataset --private {dataset_name} {output_dir}"

# コマンドの実行
result = subprocess.run(upload_command, shell=True, capture_output=True, text=True)

# 実行結果の表示
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

if result.returncode == 0:
    print(f"Hugging FaceにJSON形式のデータセットがアップロードされました: {dataset_name}")
else:
    print(f"エラーが発生しました: {result.stderr}")

STDOUT: https://huggingface.co/datasets/geniacllm/jp-effective-instructions-unified-v1/tree/main/.

STDERR: Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.

Hugging FaceにJSON形式のデータセットがアップロードされました: geniacllm/jp-effective-instructions-unified-v1


### 元データセットと新データセット比較

In [ ]:
# アップロードしたデータセットをロード
from datasets import load_dataset

dataset_org = load_dataset(f"{original_dataset_user}/{original_dataset_name}")
print("【オリジナル】")
print(dataset_org["train"][0])
print(dataset_org)
print("-" * 20)
dataset_new = load_dataset(f"{dataset_name}")
print("【変更後】")
print(dataset_new["train"][0])
print(dataset_new)

【オリジナル】
{'index': '0', 'input': 'nan', 'instruction': 'あなたは私にいくつかのコンピューターの制作プログラムを与えることができます、それは無料です', 'category': 'osaat1', 'output': 'はい、コンピューター用の無料の音楽制作プログラムがいくつかあります。そのうちの1つは、オーディオの録音と編集に使用されるオーディシー、無料のオープンソースソフトウェアです。\n\nオーディシーの特性が含まれています。\n\nREAL -TIMEオーディオ録画：マイクまたはその他の入力ソースを介してライブオーディオを録画できます。\n\nオーディオエディション：オーディオセクションの切断、コピー、貼り付け、オーディオの速度とトーンの変更、残響、エコー、増幅などの効果の適用など、オーディオを編集するためのさまざまなツールを提供します。\n\nMultitrack録画：いくつかのオーディオの手がかりを同時に録音して混ぜることができます。\n\nオーディオファイルのインポートとエクスポート：WAV、MP3、AIFFなどの多数のオーディオファイル形式と互換性があり、MP3やOGGなどのフォーマットにオーディオファイルをエクスポートできます。\n\nエフェクトプラグイン：イコライザー、コンプレッサー、リミッターなど、さまざまなオーディオエフェクトプラグインが含まれています。\n\nカスタマイズ可能なユーザーインターフェイス：ユーザー設定に適応する簡単な使用およびカスタマイズ可能なユーザーインターフェイスを提供します。'}
DatasetDict({
    train: Dataset({
        features: ['index', 'input', 'instruction', 'category', 'output'],
        num_rows: 5395
    })
})
--------------------


Generating train split: 0 examples [00:00, ? examples/s]

【変更後】
{'category': 'osaat1', 'index': '0', 'output': 'はい、コンピューター用の無料の音楽制作プログラムがいくつかあります。そのうちの1つは、オーディオの録音と編集に使用されるオーディシー、無料のオープンソースソフトウェアです。\n\nオーディシーの特性が含まれています。\n\nREAL -TIMEオーディオ録画：マイクまたはその他の入力ソースを介してライブオーディオを録画できます。\n\nオーディオエディション：オーディオセクションの切断、コピー、貼り付け、オーディオの速度とトーンの変更、残響、エコー、増幅などの効果の適用など、オーディオを編集するためのさまざまなツールを提供します。\n\nMultitrack録画：いくつかのオーディオの手がかりを同時に録音して混ぜることができます。\n\nオーディオファイルのインポートとエクスポート：WAV、MP3、AIFFなどの多数のオーディオファイル形式と互換性があり、MP3やOGGなどのフォーマットにオーディオファイルをエクスポートできます。\n\nエフェクトプラグイン：イコライザー、コンプレッサー、リミッターなど、さまざまなオーディオエフェクトプラグインが含まれています。\n\nカスタマイズ可能なユーザーインターフェイス：ユーザー設定に適応する簡単な使用およびカスタマイズ可能なユーザーインターフェイスを提供します。', 'input': '', 'instruction': 'あなたは私にいくつかのコンピューターの制作プログラムを与えることができます、それは無料です'}
DatasetDict({
    train: Dataset({
        features: ['category', 'index', 'output', 'input', 'instruction'],
        num_rows: 5395
    })
})


## kunishou/amenokaku-code-instruct

### データ加工・保存（ローカル）

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import math
import os
import json
import subprocess

original_dataset_user = "kunishou"
original_dataset_name = "amenokaku-code-instruct"
up_version = 1  # データセットのバージョン
output_dir = original_dataset_name

# データセットの読み込み読み込み
dataset = load_dataset(f"{original_dataset_user}/{original_dataset_name}")

print(dataset)

# def unify_format(example):
#     return example

# unified_dataset = {}
# # データセットに対してフォーマットの統一を適用
# for split in dataset.keys():
#     unified_split = dataset[split].map(unify_format)
#     unified_dataset[split] = unified_split

# 統一されたデータセットをJSON形式で保存
if not os.path.exists(original_dataset_name):
    os.makedirs(original_dataset_name)

for split in unified_dataset.keys():
    json_path = f"{output_dir}/{original_dataset_name}_{split}.json"
    data = unified_dataset[split]
    # データをリストとして保存
    data_list = [data[i] for i in range(len(data))]
    # 項目のキーの順序を指定して保存
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"新しいJSON形式のデータセットが作成されました: {original_dataset_name}")

Generating train split:   0%|          | 0/5395 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'input', 'instruction', 'category', 'output'],
        num_rows: 5395
    })
})


Map:   0%|          | 0/5395 [00:00<?, ? examples/s]

新しいJSON形式のデータセットが作成されました: jp-effective-instructions


### HuggingFaceへのアップロード

In [ ]:
# コマンドを構築
dataset_name = f"{organization_name}/{original_dataset_name}-unified-v{up_version}"
upload_command = f"huggingface-cli upload --repo-type=dataset --private {dataset_name} {output_dir}"

# コマンドの実行
result = subprocess.run(upload_command, shell=True, capture_output=True, text=True)

# 実行結果の表示
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

if result.returncode == 0:
    print(f"Hugging FaceにJSON形式のデータセットがアップロードされました: {dataset_name}")
else:
    print(f"エラーが発生しました: {result.stderr}")

STDOUT: https://huggingface.co/datasets/geniacllm/jp-effective-instructions-unified-v1/tree/main/.

STDERR: Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.

Hugging FaceにJSON形式のデータセットがアップロードされました: geniacllm/jp-effective-instructions-unified-v1


### 元データセットと新データセット比較

In [ ]:
# アップロードしたデータセットをロード
from datasets import load_dataset

dataset_org = load_dataset(f"{original_dataset_user}/{original_dataset_name}")
print("【オリジナル】")
print(dataset_org["train"][0])
print(dataset_org)
print("-" * 20)
dataset_new = load_dataset(f"{dataset_name}")
print("【変更後】")
print(dataset_new["train"][0])
print(dataset_new)

【オリジナル】
{'index': '0', 'input': 'nan', 'instruction': 'あなたは私にいくつかのコンピューターの制作プログラムを与えることができます、それは無料です', 'category': 'osaat1', 'output': 'はい、コンピューター用の無料の音楽制作プログラムがいくつかあります。そのうちの1つは、オーディオの録音と編集に使用されるオーディシー、無料のオープンソースソフトウェアです。\n\nオーディシーの特性が含まれています。\n\nREAL -TIMEオーディオ録画：マイクまたはその他の入力ソースを介してライブオーディオを録画できます。\n\nオーディオエディション：オーディオセクションの切断、コピー、貼り付け、オーディオの速度とトーンの変更、残響、エコー、増幅などの効果の適用など、オーディオを編集するためのさまざまなツールを提供します。\n\nMultitrack録画：いくつかのオーディオの手がかりを同時に録音して混ぜることができます。\n\nオーディオファイルのインポートとエクスポート：WAV、MP3、AIFFなどの多数のオーディオファイル形式と互換性があり、MP3やOGGなどのフォーマットにオーディオファイルをエクスポートできます。\n\nエフェクトプラグイン：イコライザー、コンプレッサー、リミッターなど、さまざまなオーディオエフェクトプラグインが含まれています。\n\nカスタマイズ可能なユーザーインターフェイス：ユーザー設定に適応する簡単な使用およびカスタマイズ可能なユーザーインターフェイスを提供します。'}
DatasetDict({
    train: Dataset({
        features: ['index', 'input', 'instruction', 'category', 'output'],
        num_rows: 5395
    })
})
--------------------


Generating train split: 0 examples [00:00, ? examples/s]

【変更後】
{'category': 'osaat1', 'index': '0', 'output': 'はい、コンピューター用の無料の音楽制作プログラムがいくつかあります。そのうちの1つは、オーディオの録音と編集に使用されるオーディシー、無料のオープンソースソフトウェアです。\n\nオーディシーの特性が含まれています。\n\nREAL -TIMEオーディオ録画：マイクまたはその他の入力ソースを介してライブオーディオを録画できます。\n\nオーディオエディション：オーディオセクションの切断、コピー、貼り付け、オーディオの速度とトーンの変更、残響、エコー、増幅などの効果の適用など、オーディオを編集するためのさまざまなツールを提供します。\n\nMultitrack録画：いくつかのオーディオの手がかりを同時に録音して混ぜることができます。\n\nオーディオファイルのインポートとエクスポート：WAV、MP3、AIFFなどの多数のオーディオファイル形式と互換性があり、MP3やOGGなどのフォーマットにオーディオファイルをエクスポートできます。\n\nエフェクトプラグイン：イコライザー、コンプレッサー、リミッターなど、さまざまなオーディオエフェクトプラグインが含まれています。\n\nカスタマイズ可能なユーザーインターフェイス：ユーザー設定に適応する簡単な使用およびカスタマイズ可能なユーザーインターフェイスを提供します。', 'input': '', 'instruction': 'あなたは私にいくつかのコンピューターの制作プログラムを与えることができます、それは無料です'}
DatasetDict({
    train: Dataset({
        features: ['category', 'index', 'output', 'input', 'instruction'],
        num_rows: 5395
    })
})


## CohereForAI/aya_collection_language_split(japanese)

### データ加工・保存（ローカル）

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import math
import os
import json
import subprocess

original_dataset_user = "CohereForAI"
original_dataset_name = "aya_collection_language_split"
up_version = 1  # データセットのバージョン
output_dir = original_dataset_name

# データセットの読み込み読み込み
dataset = load_dataset(f"{original_dataset_user}/{original_dataset_name}", "japanese")

print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/6218459 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/295333 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/305786 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'inputs', 'targets', 'dataset_name', 'sub_dataset_name', 'task_type', 'template_id', 'language', 'script', 'split'],
        num_rows: 6218459
    })
    validation: Dataset({
        features: ['id', 'inputs', 'targets', 'dataset_name', 'sub_dataset_name', 'task_type', 'template_id', 'language', 'script', 'split'],
        num_rows: 295333
    })
    test: Dataset({
        features: ['id', 'inputs', 'targets', 'dataset_name', 'sub_dataset_name', 'task_type', 'template_id', 'language', 'script', 'split'],
        num_rows: 305786
    })
})


In [ ]:
def unify_format(example):
    # カラム名を変更
    example['instruction'] = example.pop('inputs')
    example['output'] = example.pop('targets')

    return example

unified_dataset = {}
# データセットに対してフォーマットの統一を適用
for split in dataset.keys():
    unified_split = dataset[split].map(unify_format)
    unified_dataset[split] = unified_split

# 統一されたデータセットをJSON形式で保存
if not os.path.exists(original_dataset_name):
    os.makedirs(original_dataset_name)

for split in unified_dataset.keys():
    json_path = f"{output_dir}/{original_dataset_name}_{split}.json"
    data = unified_dataset[split]
    # データをリストとして保存
    data_list = [data[i] for i in range(len(data))]
    # 項目のキーの順序を指定して保存
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"新しいJSON形式のデータセットが作成されました: {original_dataset_name}")

Map:   0%|          | 0/6218459 [00:00<?, ? examples/s]

Map:   0%|          | 0/295333 [00:00<?, ? examples/s]

Map:   0%|          | 0/305786 [00:00<?, ? examples/s]

新しいJSON形式のデータセットが作成されました: aya_collection_language_split


### HuggingFaceへのアップロード

In [ ]:
# json形式確認
import json

def check_json_format(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"{file_path} is a valid JSON file.")
    except json.JSONDecodeError as e:
        print(f"Error in {file_path}: {e}")

json_path = f"{output_dir}/{original_dataset_name}_train.json"
check_json_format(json_path)

aya_collection_language_split/aya_collection_language_split_train.json is a valid JSON file.


In [ ]:
# コマンドを構築
dataset_name = f"{organization_name}/{original_dataset_name}-unified-v{up_version}"
upload_command = f"huggingface-cli upload --repo-type=dataset --private {dataset_name} {output_dir}"

# コマンドの実行
result = subprocess.run(upload_command, shell=True, capture_output=True, text=True)

# 実行結果の表示
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

if result.returncode == 0:
    print(f"Hugging FaceにJSON形式のデータセットがアップロードされました: {dataset_name}")
else:
    print(f"エラーが発生しました: {result.stderr}")

STDOUT: https://huggingface.co/datasets/geniacllm/aya_collection_language_split-unified-v1/tree/main/.

STDERR: Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.

aya_collection_language_split_test.json:   0%|          | 0.00/273M [00:00<?, ?B/s]


Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

aya_collection_language_split_train.json:   0%|          | 0.00/4.85G [00:00<?, ?B/s]



aya_collection_language_split_validation.json:   0%|          | 0.00/249M [00:00<?, ?B/s]



aya_collection_language_split_validation.json:   0%|          | 16.4k/249M [00:00<30:45, 135kB/s]
aya_collection_language_split_test.json:   0%|          | 16.4k/273M [00:00<39:06, 116kB/s]

aya_collection_language_split_train.json:   0%|          | 16.4k/4.85G [00:00<11:22:17, 118kB/s]



aya_collection_language_split_validation.json:   3%|▎         | 6.34M/249M [00:00<00:07, 34.2MB/s]


### 元データセットと新データセット比較

In [ ]:
# アップロードしたデータセットをロード
from datasets import load_dataset

dataset_org = load_dataset(f"{original_dataset_user}/{original_dataset_name}", "japanese")
print("【オリジナル】")
print(dataset_org["train"][0])
print(dataset_org["validation"][0])
print(dataset_org["test"][0])
print(dataset_org)
print("-" * 20)
dataset_new = load_dataset(f"{dataset_name}")
print("【変更後】")
print(dataset_new["train"][0])
print(dataset_new["validation"][0])
print(dataset_new["test"][0])
print(dataset_new)

【オリジナル】
{'id': 1, 'inputs': 'トルコ人 (),またはトルコ人 (),またアナトリア人 (),は,主にトルコに住み,最も広く話されているトルコ語であるトルコ語を話しているトルコ人の民族集団と民族である. 彼らはトルコ最大の民族集団であり,トルコ語話者のうちでも最大の民族集団である. オスマン帝国の旧領にはトルコ人少数民族がいます. さらに,特に西ヨーロッパでは,近代的な移住によってトルコ人 डायアスポラが確立した.  "トルコ人"という民族名はヘロドトスの (c. 紀元前484~425年) はスキーテスの初代王ターギタスに言及している.さらに,紀元後1世紀にポンポニウス・メラはアゾフ海北部の森林の"ターケ"を言及し,プリニウス大王は同じ地域の住民の"ティルケ"をリストに挙げた. "トルコ人"に関する最初の明確な言及は,主に6世紀の中国の情報源から来ている. これらの情報源では",トルコ人"は"トゥジュエ" (Tujue) として登場し,ゴクトゥルク族を指している. "トルコ人"はトルコ人を指すが,時としてトルコ人の言語集団を指す.  19世紀には"トルコ人"という言葉は アナトリアの村人にだけ使われていた. オスマン帝国の支配階級は オスマン人として自己認識し 通常はトルコ人として自己認識しなかった. 19世紀後半に オスマン帝国の上流階級が 民族主義のヨーロッパ思想を採用し "トルコ人"という用語は より積極的な意味合いを持つようになりました アナトリアのトルコ語話者は オスマン帝国の支配を最も忠実に支持した.  次の質問に答えなさい 1 について 19世紀には"トルコ人"という言葉が何を指していたのか? 2 ほか の 人 19世紀半ばには 支配階級が自らを表現するのに この言葉を使っていたのでしょうか? 3 について 時代が経つにつれ この用語は より好ましいものとして見られたか? 4. 薬剤 について トルコの最大の民族は? 5 について トルコ人はみんなトルコに居たのか? 6 について 現在,移住が人気のある地域は? 7 について スキテヤ 人 の 最初 の 王 は だれ でし た か 8 について ヘロドトスはいつ生きたのか. 9 について トルコの人々を指す別の用語は? 10 歳 彼らは一般的にどんな言語を話しますか. 11 

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

【変更後】
{'split': 'train', 'sub_dataset_name': '-', 'script': 'Jpan', 'dataset_name': 'Flan-Coqa (T)', 'language': 'jpn', 'output': '1 について アナトリアの村人 2 ほか の 人 違う 3 について そうだ 4. トルコ人 5 について 違う 6 について 西ヨーロッパ 7 について ターギタス 8 について 紀元前484~425年 9 について アナトリアのトルコ人 10 歳 トルコ語 11 歳 そうだ', 'id': 1, 'task_type': 'question-answering', 'template_id': 1, 'instruction': 'トルコ人 (),またはトルコ人 (),またアナトリア人 (),は,主にトルコに住み,最も広く話されているトルコ語であるトルコ語を話しているトルコ人の民族集団と民族である. 彼らはトルコ最大の民族集団であり,トルコ語話者のうちでも最大の民族集団である. オスマン帝国の旧領にはトルコ人少数民族がいます. さらに,特に西ヨーロッパでは,近代的な移住によってトルコ人 डायアスポラが確立した.  "トルコ人"という民族名はヘロドトスの (c. 紀元前484~425年) はスキーテスの初代王ターギタスに言及している.さらに,紀元後1世紀にポンポニウス・メラはアゾフ海北部の森林の"ターケ"を言及し,プリニウス大王は同じ地域の住民の"ティルケ"をリストに挙げた. "トルコ人"に関する最初の明確な言及は,主に6世紀の中国の情報源から来ている. これらの情報源では",トルコ人"は"トゥジュエ" (Tujue) として登場し,ゴクトゥルク族を指している. "トルコ人"はトルコ人を指すが,時としてトルコ人の言語集団を指す.  19世紀には"トルコ人"という言葉は アナトリアの村人にだけ使われていた. オスマン帝国の支配階級は オスマン人として自己認識し 通常はトルコ人として自己認識しなかった. 19世紀後半に オスマン帝国の上流階級が 民族主義のヨーロッパ思想を採用し "トルコ人"という用語は より積極的な意味合いを持つようになりました アナトリアのトルコ語話者は オスマン帝国の支配を最も忠実に支持

## databricks/databricks-dolly-15k

### データ加工・保存（ローカル）

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import math
import os
import json
import subprocess

original_dataset_user = "databricks"
original_dataset_name = "databricks-dolly-15k"
up_version = 1  # データセットのバージョン
output_dir = original_dataset_name

# データセットの読み込み読み込み
dataset = load_dataset(f"{original_dataset_user}/{original_dataset_name}")

print(dataset)


Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})


In [ ]:
def unify_format(example):
    # カラム名を変更
    example["input"] = example.pop("context")
    example["output"] = example.pop("response")

    return example

unified_dataset = {}
# データセットに対してフォーマットの統一を適用
for split in dataset.keys():
    unified_split = dataset[split].map(unify_format)
    unified_dataset[split] = unified_split

# 統一されたデータセットをJSON形式で保存
if not os.path.exists(original_dataset_name):
    os.makedirs(original_dataset_name)

for split in unified_dataset.keys():
    json_path = f"{output_dir}/{original_dataset_name}_{split}.json"
    data = unified_dataset[split]
    # データをリストとして保存
    data_list = [data[i] for i in range(len(data))]
    # 項目のキーの順序を指定して保存
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"新しいJSON形式のデータセットが作成されました: {original_dataset_name}")

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

新しいJSON形式のデータセットが作成されました: databricks-dolly-15k


### HuggingFaceへのアップロード

In [ ]:
# json形式確認
import json

def check_json_format(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"{file_path} is a valid JSON file.")
    except json.JSONDecodeError as e:
        print(f"Error in {file_path}: {e}")

json_path = f"{output_dir}/{original_dataset_name}_train.json"
check_json_format(json_path)

databricks-dolly-15k/databricks-dolly-15k_train.json is a valid JSON file.


In [ ]:
# コマンドを構築
dataset_name = f"{organization_name}/{original_dataset_name}-unified-v{up_version}"
upload_command = f"huggingface-cli upload --repo-type=dataset --private {dataset_name} {output_dir}"

# コマンドの実行
result = subprocess.run(upload_command, shell=True, capture_output=True, text=True)

# 実行結果の表示
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

if result.returncode == 0:
    print(f"Hugging FaceにJSON形式のデータセットがアップロードされました: {dataset_name}")
else:
    print(f"エラーが発生しました: {result.stderr}")

STDOUT: https://huggingface.co/datasets/geniacllm/databricks-dolly-15k-unified-v1/tree/main/.

STDERR: Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.

databricks-dolly-15k_train.json:   0%|          | 0.00/13.6M [00:00<?, ?B/s]
databricks-dolly-15k_train.json:   1%|          | 164k/13.6M [00:00<00:09, 1.41MB/s]
databricks-dolly-15k_train.json:  61%|██████▏   | 8.34M/13.6M [00:00<00:00, 42.6MB/s]
databricks-dolly-15k_train.json: 100%|██████████| 13.6M/13.6M [00:00<00:00, 25.5MB/s]

Hugging FaceにJSON形式のデータセットがアップロードされました: geniacllm/databricks-dolly-15k-unified-v1


### 元データセットと新データセット比較

In [ ]:
# アップロードしたデータセットをロード
from datasets import load_dataset

dataset_org = load_dataset(f"{original_dataset_user}/{original_dataset_name}")
print("【オリジナル】")
print(dataset_org["train"][0])
print(dataset_org)
print("-" * 20)
dataset_new = load_dataset(f"{dataset_name}")
print("【変更後】")
print(dataset_new["train"][0])
print(dataset_new)

【オリジナル】
{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}
DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 15011
    })
})
--------------------


Generating train split: 0 examples [00:00, ? examples/s]

【変更後】
{'instruction': 'When did Virgin Australia start operating?', 'input': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'category': 'closed_qa', 'output': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.'}
DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'category', 'output'],
        num_rows: 15011
    })
})


## microsoft/orca-math-word-problems-200k

### データ加工・保存（ローカル）

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import math
import os
import json
import subprocess

original_dataset_user = "microsoft"
original_dataset_name = "orca-math-word-problems-200k"
up_version = 1  # データセットのバージョン
output_dir = original_dataset_name

# データセットの読み込み読み込み
dataset = load_dataset(f"{original_dataset_user}/{original_dataset_name}")

print(dataset)


Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 200035
    })
})


In [ ]:
def unify_format(example):
    # カラム名を変更
    example["instruction"] = example.pop("question")
    example["output"] = example.pop("answer")

    return example

unified_dataset = {}
# データセットに対してフォーマットの統一を適用
for split in dataset.keys():
    unified_split = dataset[split].map(unify_format)
    unified_dataset[split] = unified_split

# 統一されたデータセットをJSON形式で保存
if not os.path.exists(original_dataset_name):
    os.makedirs(original_dataset_name)

for split in unified_dataset.keys():
    json_path = f"{output_dir}/{original_dataset_name}_{split}.json"
    data = unified_dataset[split]
    # データをリストとして保存
    data_list = [data[i] for i in range(len(data))]
    # 項目のキーの順序を指定して保存
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"新しいJSON形式のデータセットが作成されました: {original_dataset_name}")

Map:   0%|          | 0/200035 [00:00<?, ? examples/s]

新しいJSON形式のデータセットが作成されました: orca-math-word-problems-200k


### HuggingFaceへのアップロード

In [ ]:
# json形式確認
import json

def check_json_format(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"{file_path} is a valid JSON file.")
    except json.JSONDecodeError as e:
        print(f"Error in {file_path}: {e}")

json_path = f"{output_dir}/{original_dataset_name}_train.json"
check_json_format(json_path)

orca-math-word-problems-200k/orca-math-word-problems-200k_train.json is a valid JSON file.


In [ ]:
# コマンドを構築
dataset_name = f"{organization_name}/{original_dataset_name}-unified-v{up_version}"
upload_command = f"huggingface-cli upload --repo-type=dataset --private {dataset_name} {output_dir}"

# コマンドの実行
result = subprocess.run(upload_command, shell=True, capture_output=True, text=True)

# 実行結果の表示
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

if result.returncode == 0:
    print(f"Hugging FaceにJSON形式のデータセットがアップロードされました: {dataset_name}")
else:
    print(f"エラーが発生しました: {result.stderr}")

STDOUT: https://huggingface.co/datasets/geniacllm/orca-math-word-problems-200k-unified-v1/tree/main/.

STDERR: Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.

orca-math-word-problems-200k_train.json:   0%|          | 0.00/241M [00:00<?, ?B/s]
orca-math-word-problems-200k_train.json:   0%|          | 262k/241M [00:00<01:37, 2.48MB/s]
orca-math-word-problems-200k_train.json:   4%|▎         | 8.93M/241M [00:00<00:04, 48.9MB/s]
orca-math-word-problems-200k_train.json:   7%|▋         | 16.0M/241M [00:00<00:07, 29.4MB/s]
orca-math-word-problems-200k_train.json:  11%|█         | 26.6M/241M [00:00<00:04, 46.2MB/s]
orca-math-word-problems-200k_train.json:  13%|█▎        | 32.5M/241M [00:00<00:06, 32.3MB/s]
orca-math-word-problems-200k_train.json:  18%|█▊        | 43.4M/241M [00:01<00:04, 46.0MB/s]
orca-math-word-problems-200k_train.json:  21%|██        | 49.7M/241M [00:01<0

### 元データセットと新データセット比較

In [ ]:
# アップロードしたデータセットをロード
from datasets import load_dataset

dataset_org = load_dataset(f"{original_dataset_user}/{original_dataset_name}")
print("【オリジナル】")
print(dataset_org["train"][0])
print(dataset_org)
print("-" * 20)
dataset_new = load_dataset(f"{dataset_name}")
print("【変更後】")
print(dataset_new["train"][0])
print(dataset_new)

【オリジナル】
{'question': 'Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.', 'answer': 'If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.'}
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 200035
    })
})
--------------------


Generating train split: 0 examples [00:00, ? examples/s]

【変更後】
{'instruction': 'Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.', 'output': 'If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.'}
DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 200035
    })
})


## kunishou/OpenMathInstruct-1-1.8m-ja

### データ加工・保存（ローカル）

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import math
import os
import json
import subprocess

original_dataset_user = "kunishou"
original_dataset_name = "OpenMathInstruct-1-1.8m-ja"
up_version = 1  # データセットのバージョン
output_dir = original_dataset_name

# データセットの読み込み読み込み
dataset = load_dataset(f"{original_dataset_user}/{original_dataset_name}")

print(dataset)


Generating train split:   0%|          | 0/1825008 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'dataset', 'question', 'generated_solution', 'question_ja', 'generated_solution_ja'],
        num_rows: 1825008
    })
})


In [ ]:
def unify_format(example):
    # カラム名を変更
    example["instruction"] = example.pop("question_ja")
    example["output"] = example.pop("generated_solution_ja")

    return example

unified_dataset = {}
# データセットに対してフォーマットの統一を適用
for split in dataset.keys():
    unified_split = dataset[split].map(unify_format)
    unified_dataset[split] = unified_split

# 統一されたデータセットをJSON形式で保存
if not os.path.exists(original_dataset_name):
    os.makedirs(original_dataset_name)

for split in unified_dataset.keys():
    json_path = f"{output_dir}/{original_dataset_name}_{split}.json"
    data = unified_dataset[split]
    # データをリストとして保存
    data_list = [data[i] for i in range(len(data))]
    # 項目のキーの順序を指定して保存
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"新しいJSON形式のデータセットが作成されました: {original_dataset_name}")

Map:   0%|          | 0/1825008 [00:00<?, ? examples/s]

新しいJSON形式のデータセットが作成されました: OpenMathInstruct-1-1.8m-ja


### HuggingFaceへのアップロード

In [ ]:
# json形式確認
import json

def check_json_format(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"{file_path} is a valid JSON file.")
    except json.JSONDecodeError as e:
        print(f"Error in {file_path}: {e}")

json_path = f"{output_dir}/{original_dataset_name}_train.json"
check_json_format(json_path)

OpenMathInstruct-1-1.8m-ja/OpenMathInstruct-1-1.8m-ja_train.json is a valid JSON file.


In [ ]:
# コマンドを構築
dataset_name = f"{organization_name}/{original_dataset_name}-unified-v{up_version}"
upload_command = f"huggingface-cli upload --repo-type=dataset --private {dataset_name} {output_dir}"

# コマンドの実行
result = subprocess.run(upload_command, shell=True, capture_output=True, text=True)

# 実行結果の表示
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

if result.returncode == 0:
    print(f"Hugging FaceにJSON形式のデータセットがアップロードされました: {dataset_name}")
else:
    print(f"エラーが発生しました: {result.stderr}")

STDOUT: https://huggingface.co/datasets/geniacllm/OpenMathInstruct-1-1.8m-ja-unified-v1/tree/main/.

STDERR: Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.

OpenMathInstruct-1-1.8m-ja_train.json:   0%|          | 0.00/2.87G [00:00<?, ?B/s]
OpenMathInstruct-1-1.8m-ja_train.json:   0%|          | 377k/2.87G [00:00<13:18, 3.60MB/s]
OpenMathInstruct-1-1.8m-ja_train.json:   0%|          | 9.31M/2.87G [00:00<00:57, 49.7MB/s]
OpenMathInstruct-1-1.8m-ja_train.json:   1%|          | 16.0M/2.87G [00:00<01:20, 35.4MB/s]
OpenMathInstruct-1-1.8m-ja_train.json:   1%|          | 25.5M/2.87G [00:00<00:56, 50.6MB/s]
OpenMathInstruct-1-1.8m-ja_train.json:   1%|          | 32.0M/2.87G [00:00<01:18, 36.4MB/s]
OpenMathInstruct-1-1.8m-ja_train.json:   1%|▏         | 41.3M/2.87G [00:00<00:59, 47.4MB/s]
OpenMathInstruct-1-1.8m-ja_train.json:   2%|▏         | 48.0M/2.87G [00:01<01:21, 34.6

### 元データセットと新データセット比較

In [ ]:
# アップロードしたデータセットをロード
from datasets import load_dataset

dataset_org = load_dataset(f"{original_dataset_user}/{original_dataset_name}")
print("【オリジナル】")
print(dataset_org["train"][0])
print(dataset_org)
print("-" * 20)
dataset_new = load_dataset(f"{dataset_name}")
print("【変更後】")
print(dataset_new["train"][0])
print(dataset_new)

【オリジナル】
{'index': 0, 'dataset': 'gsm8k', 'question': 'Martha has 18 crayons. She lost half of them, so she bought a new set of 20 crayons. How many crayons in total does Martha have after the purchase?', 'generated_solution': "Let's solve this problem using Python code.\n<llm-code>\namount_of_lost_crayons = 18 / 2\namount_of_new_crayons = 20\ntotal_amount = amount_of_lost_crayons + amount_of_new_crayons\ntotal_amount\n</llm-code>\n<llm-code-output>\n29.0\n</llm-code-output>\nThus Martha has \\\\boxed{29} crayons in total.", 'question_ja': 'マーサは18本のクレヨンを持っている。その半分をなくしてしまったので、新しいクレヨンを20本買いました。購入後、マーサは全部で何本のクレヨンを持っていますか。', 'generated_solution_ja': 'Pythonコードを使用してこの問題を解決しましょう。<llm-code>\namount_of_lost_crayons = 18 / 2\namount_of_new_crayons = 20\ntotal_amount = amount_of_lost_crayons + amount_of_new_crayons\ntotal_amount\n</llm-code><llm-code-output>\n29.0\n</llm-code-output>\nしたがって、マーサは合計で\\\\ boxed {29}本のクレヨンを持っています。'}
DatasetDict({
    train: Dataset({
        features: ['index', 'data

Generating train split: 0 examples [00:00, ? examples/s]

【変更後】
{'instruction': 'マーサは18本のクレヨンを持っている。その半分をなくしてしまったので、新しいクレヨンを20本買いました。購入後、マーサは全部で何本のクレヨンを持っていますか。', 'index': 0, 'dataset': 'gsm8k', 'generated_solution': "Let's solve this problem using Python code.\n<llm-code>\namount_of_lost_crayons = 18 / 2\namount_of_new_crayons = 20\ntotal_amount = amount_of_lost_crayons + amount_of_new_crayons\ntotal_amount\n</llm-code>\n<llm-code-output>\n29.0\n</llm-code-output>\nThus Martha has \\\\boxed{29} crayons in total.", 'output': 'Pythonコードを使用してこの問題を解決しましょう。<llm-code>\namount_of_lost_crayons = 18 / 2\namount_of_new_crayons = 20\ntotal_amount = amount_of_lost_crayons + amount_of_new_crayons\ntotal_amount\n</llm-code><llm-code-output>\n29.0\n</llm-code-output>\nしたがって、マーサは合計で\\\\ boxed {29}本のクレヨンを持っています。', 'question': 'Martha has 18 crayons. She lost half of them, so she bought a new set of 20 crayons. How many crayons in total does Martha have after the purchase?'}
DatasetDict({
    train: Dataset({
        features: ['instruction', 'index', 'datase

## kunishou/ApolloCorpus-ja

### データ加工・保存（ローカル）

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import math
import os
import json
import subprocess

original_dataset_user = "kunishou"
original_dataset_name = "ApolloCorpus-ja"
up_version = 1  # データセットのバージョン
output_dir = original_dataset_name

# データセットの読み込み読み込み
dataset = load_dataset(f"{original_dataset_user}/{original_dataset_name}")

print(dataset)


Generating train split:   0%|          | 0/524827 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['index', 'prompt', 'response', 'prompt_ja', 'response_ja'],
        num_rows: 524827
    })
})


In [ ]:
def unify_format(example):
    # カラム名を変更
    example["instruction"] = example.pop("prompt_ja")
    example["output"] = example.pop("response_ja")

    return example

unified_dataset = {}
# データセットに対してフォーマットの統一を適用
for split in dataset.keys():
    unified_split = dataset[split].map(unify_format)
    unified_dataset[split] = unified_split

# 統一されたデータセットをJSON形式で保存
if not os.path.exists(original_dataset_name):
    os.makedirs(original_dataset_name)

for split in unified_dataset.keys():
    json_path = f"{output_dir}/{original_dataset_name}_{split}.json"
    data = unified_dataset[split]
    # データをリストとして保存
    data_list = [data[i] for i in range(len(data))]
    # 項目のキーの順序を指定して保存
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"新しいJSON形式のデータセットが作成されました: {original_dataset_name}")

Map:   0%|          | 0/524827 [00:00<?, ? examples/s]

新しいJSON形式のデータセットが作成されました: ApolloCorpus-ja


### HuggingFaceへのアップロード

In [ ]:
# json形式確認
import json

def check_json_format(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"{file_path} is a valid JSON file.")
    except json.JSONDecodeError as e:
        print(f"Error in {file_path}: {e}")

json_path = f"{output_dir}/{original_dataset_name}_train.json"
check_json_format(json_path)

ApolloCorpus-ja/ApolloCorpus-ja_train.json is a valid JSON file.


In [ ]:
# コマンドを構築
dataset_name = f"{organization_name}/{original_dataset_name}-unified-v{up_version}"
upload_command = f"huggingface-cli upload --repo-type=dataset --private {dataset_name} {output_dir}"

# コマンドの実行
result = subprocess.run(upload_command, shell=True, capture_output=True, text=True)

# 実行結果の表示
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

if result.returncode == 0:
    print(f"Hugging FaceにJSON形式のデータセットがアップロードされました: {dataset_name}")
else:
    print(f"エラーが発生しました: {result.stderr}")

STDOUT: https://huggingface.co/datasets/geniacllm/ApolloCorpus-ja-unified-v1/tree/main/.

STDERR: Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.

ApolloCorpus-ja_train.json:   0%|          | 0.00/2.12G [00:00<?, ?B/s]
ApolloCorpus-ja_train.json:   0%|          | 262k/2.12G [00:00<14:02, 2.51MB/s]
ApolloCorpus-ja_train.json:   0%|          | 8.60M/2.12G [00:00<00:46, 45.8MB/s]
ApolloCorpus-ja_train.json:   1%|          | 16.0M/2.12G [00:00<01:08, 30.7MB/s]
ApolloCorpus-ja_train.json:   1%|          | 26.0M/2.12G [00:00<00:44, 47.4MB/s]
ApolloCorpus-ja_train.json:   2%|▏         | 32.0M/2.12G [00:00<01:09, 30.2MB/s]
ApolloCorpus-ja_train.json:   2%|▏         | 42.6M/2.12G [00:01<00:49, 42.3MB/s]
ApolloCorpus-ja_train.json:   2%|▏         | 48.5M/2.12G [00:01<01:01, 33.7MB/s]
ApolloCorpus-ja_train.json:   3%|▎         | 57.9M/2.12G [00:01<00:46, 44.2MB/s]
ApolloCorpus

### 元データセットと新データセット比較

In [ ]:
# アップロードしたデータセットをロード
from datasets import load_dataset

dataset_org = load_dataset(f"{original_dataset_user}/{original_dataset_name}")
print("【オリジナル】")
print(dataset_org["train"][0])
print(dataset_org)
print("-" * 20)
dataset_new = load_dataset(f"{dataset_name}")
print("【変更後】")
print(dataset_new["train"][0])
print(dataset_new)

【オリジナル】
{'index': 0, 'prompt': 'What are the potential applications of Li4SiO4 as a beam shaping material for boron neutron capture therapy facilities?', 'response': 'Li4SiO4 has the potential to be used as a beam shaping material for boron neutron capture therapy (BNCT) facilities. In a study evaluating its thermal neutron attenuation properties, Li4SiO4 demonstrated remarkable effectiveness compared to other neutron-absorbing materials. This means that it has the ability to effectively attenuate thermal neutrons, which is crucial for BNCT applications.\n\nThe synthesis and characterization of Li4SiO4 were performed using different methods, including x-ray, electron diffraction, and transmission electron microscopy. The neutron measurements were conducted at the BNCT and neutron radiography facilities to assess its performance.\n\nOne advantage of Li4SiO4 is its natural isotopic abundance, which contributes to its effectiveness as a beam shaping material. Additionally, the availabilit

Generating train split: 0 examples [00:00, ? examples/s]

【変更後】
{'instruction': 'ホウ素中性子捕獲療法施設のビーム型材料としてのLi4Sio4の潜在的な応用は何ですか？', 'index': 0, 'response': 'Li4SiO4 has the potential to be used as a beam shaping material for boron neutron capture therapy (BNCT) facilities. In a study evaluating its thermal neutron attenuation properties, Li4SiO4 demonstrated remarkable effectiveness compared to other neutron-absorbing materials. This means that it has the ability to effectively attenuate thermal neutrons, which is crucial for BNCT applications.\n\nThe synthesis and characterization of Li4SiO4 were performed using different methods, including x-ray, electron diffraction, and transmission electron microscopy. The neutron measurements were conducted at the BNCT and neutron radiography facilities to assess its performance.\n\nOne advantage of Li4SiO4 is its natural isotopic abundance, which contributes to its effectiveness as a beam shaping material. Additionally, the availability of natural lithium in local salt mines and the feasibility of scalable 

## alfredplpl/wikipedia-qa-ja-1m

### データ加工・保存（ローカル）

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import math
import os
import json
import subprocess

original_dataset_user = "alfredplpl"
original_dataset_name = "wikipedia-qa-ja-1m"
up_version = 1  # データセットのバージョン
output_dir = original_dataset_name

# データセットの読み込み読み込み
dataset = load_dataset(f"{original_dataset_user}/{original_dataset_name}")

print(dataset)


Generating train split:   0%|          | 0/1395760 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'question', 'answer'],
        num_rows: 1395760
    })
})


In [ ]:
def unify_format(example):
    # カラム名を変更
    example["instruction"] = example.pop("question")
    example["output"] = example.pop("answer")

    return example

unified_dataset = {}
# データセットに対してフォーマットの統一を適用
for split in dataset.keys():
    unified_split = dataset[split].map(unify_format)
    unified_dataset[split] = unified_split

# 統一されたデータセットをJSON形式で保存
if not os.path.exists(original_dataset_name):
    os.makedirs(original_dataset_name)

for split in unified_dataset.keys():
    json_path = f"{output_dir}/{original_dataset_name}_{split}.json"
    data = unified_dataset[split]
    # データをリストとして保存
    data_list = [data[i] for i in range(len(data))]
    # 項目のキーの順序を指定して保存
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data_list, f, ensure_ascii=False, indent=4)

print(f"新しいJSON形式のデータセットが作成されました: {original_dataset_name}")

Map:   0%|          | 0/1395760 [00:00<?, ? examples/s]

新しいJSON形式のデータセットが作成されました: wikipedia-qa-ja-1m


### HuggingFaceへのアップロード

In [ ]:
# json形式確認
import json

def check_json_format(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"{file_path} is a valid JSON file.")
    except json.JSONDecodeError as e:
        print(f"Error in {file_path}: {e}")

json_path = f"{output_dir}/{original_dataset_name}_train.json"
check_json_format(json_path)

wikipedia-qa-ja-1m/wikipedia-qa-ja-1m_train.json is a valid JSON file.


In [ ]:
# コマンドを構築
dataset_name = f"{organization_name}/{original_dataset_name}-unified-v{up_version}"
upload_command = f"huggingface-cli upload --repo-type=dataset --private {dataset_name} {output_dir}"

# コマンドの実行
result = subprocess.run(upload_command, shell=True, capture_output=True, text=True)

# 実行結果の表示
print("STDOUT:", result.stdout)
print("STDERR:", result.stderr)

if result.returncode == 0:
    print(f"Hugging FaceにJSON形式のデータセットがアップロードされました: {dataset_name}")
else:
    print(f"エラーが発生しました: {result.stderr}")

STDOUT: https://huggingface.co/datasets/geniacllm/wikipedia-qa-ja-1m-unified-v1/tree/main/.

STDERR: Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.

wikipedia-qa-ja-1m_train.json:   0%|          | 0.00/496M [00:00<?, ?B/s]
wikipedia-qa-ja-1m_train.json:   0%|          | 426k/496M [00:00<02:08, 3.87MB/s]
wikipedia-qa-ja-1m_train.json:   2%|▏         | 8.60M/496M [00:00<00:10, 47.4MB/s]
wikipedia-qa-ja-1m_train.json:   3%|▎         | 16.0M/496M [00:00<00:16, 28.5MB/s]
wikipedia-qa-ja-1m_train.json:   5%|▌         | 25.3M/496M [00:00<00:11, 42.6MB/s]
wikipedia-qa-ja-1m_train.json:   6%|▋         | 32.0M/496M [00:00<00:12, 36.1MB/s]
wikipedia-qa-ja-1m_train.json:   8%|▊         | 41.5M/496M [00:01<00:09, 47.9MB/s]
wikipedia-qa-ja-1m_train.json:  10%|▉         | 48.0M/496M [00:01<00:14, 30.8MB/s]
wikipedia-qa-ja-1m_train.json:  12%|█▏        | 58.3M/496M [00:01<00:10, 4

### 元データセットと新データセット比較

In [ ]:
# アップロードしたデータセットをロード
from datasets import load_dataset

dataset_org = load_dataset(f"{original_dataset_user}/{original_dataset_name}")
print("【オリジナル】")
print(dataset_org["train"][0])
print(dataset_org)
print("-" * 20)
dataset_new = load_dataset(f"{dataset_name}")
print("【変更後】")
print(dataset_new["train"][0])
print(dataset_new)

【オリジナル】
{'id': '5', 'url': 'https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%B3%E3%83%91%E3%82%B5%E3%83%B3%E3%83%89', 'question': 'アンパサンドとはなんですか？', 'answer': '並立助詞「…と…」'}
DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'question', 'answer'],
        num_rows: 1395760
    })
})
--------------------


Generating train split: 0 examples [00:00, ? examples/s]

【変更後】
{'id': '5', 'instruction': 'アンパサンドとはなんですか？', 'url': 'https://ja.wikipedia.org/wiki/%E3%82%A2%E3%83%B3%E3%83%91%E3%82%B5%E3%83%B3%E3%83%89', 'output': '並立助詞「…と…」'}
DatasetDict({
    train: Dataset({
        features: ['id', 'instruction', 'url', 'output'],
        num_rows: 1395760
    })
})
